In [ ]:
import pandas as pd

df = pd.read_csv("Crop_recommendation.csv", delimiter=';')

print("Preview data:\n", df.head())

print("\nTipe data tiap kolom:\n", df.dtypes)

print("\nJumlah data kosong per kolom:\n", df.isnull().sum())


Preview data:
     N   P   K    temperature       humidity             ph       rainfall  \
0  90  42  43  2.087.974.371  8.200.274.423  6.502.985.292  2.029.355.362   
1  85  58  41  2.177.046.169  8.031.964.408  7.038.096.361  2.266.555.374   
2  60  55  44  2.300.445.915    823.207.629  7.840.207.144  2.639.642.476   
3  74  35  40  2.649.109.635  8.015.836.264  6.980.400.905  2.428.640.342   
4  78  42  42  2.013.017.482  8.160.487.287  7.628.472.891  2.627.173.405   

  label  
0  rice  
1  rice  
2  rice  
3  rice  
4  rice  

Tipe data tiap kolom:
 N               int64
P               int64
K               int64
temperature    object
humidity       object
ph             object
rainfall       object
label          object
dtype: object

Jumlah data kosong per kolom:
 N              0
P              0
K              0
temperature    0
humidity       0
ph             0
rainfall       0
label          0
dtype: int64


In [ ]:
cols_to_clean = ['temperature', 'humidity', 'ph', 'rainfall']

for col in cols_to_clean:
    df[col] = df[col].astype(str).str.replace('.', '', regex=False).astype(float)

print(df[cols_to_clean].head())
print("\nTipe data setelah dibersihkan:\n", df[cols_to_clean].dtypes)


    temperature      humidity            ph      rainfall
0  2.087974e+10  8.200274e+10  6.502985e+10  2.029355e+10
1  2.177046e+10  8.031964e+10  7.038096e+10  2.266555e+10
2  2.300446e+10  8.232076e+09  7.840207e+10  2.639642e+10
3  2.649110e+10  8.015836e+10  6.980401e+10  2.428640e+10
4  2.013017e+10  8.160487e+10  7.628473e+10  2.627173e+10

Tipe data setelah dibersihkan:
 temperature    float64
humidity       float64
ph             float64
rainfall       float64
dtype: object


In [ ]:
print(df['label'].value_counts())

label
rice          139
Soyabeans     130
banana        130
beans         125
cowpeas       122
orange        122
maize         119
coffee        110
peas          100
groundnuts    100
mango         100
watermelon    100
grapes        100
apple         100
cotton        100
Name: count, dtype: int64


In [ ]:
X = df.drop('label', axis=1)
y = df['label']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
import pandas as pd

data_baru = pd.DataFrame([{
    "N": 90,
    "P": 42,
    "K": 43,
    "temperature": 20.87,
    "humidity": 82.00,
    "ph": 6.50,
    "rainfall": 20.29
}])

hasil_prediksi = model.predict(data_baru)
print("Rekomendasi Tanaman:", hasil_prediksi[0])

Rekomendasi Tanaman: rice
